# Corpus Processing

In [31]:
import pandas as pd
import json
from tqdm import tqdm
tqdm.pandas()
import ast

In [37]:
# directory with corpus, one sentence per line
data_ac="indeed_jobs_extractions"

# Keyword Extraction

In [48]:
# src directory
kb_base_dir="/mnt/efs/shared/meg_shared_scripts/meg-kb"

In [49]:
%cd $kb_base_dir/src/keyword_extraction/

/mnt/efs/shared/meg_shared_scripts/meg-kb/src/keyword_extraction


In [50]:
!chmod +x ./corpusProcess.sh

In [55]:
thread = 8
!./corpusProcess.sh $data_ac $thread

/mnt/efs/shared/meg_shared_scripts/meg-kb/src/keyword_extraction
===Corpus Name: indeed_jobs_extractions===
===Current Path: /mnt/efs/shared/meg_shared_scripts/meg-kb/src/keyword_extraction===
===Cleaning input corpus===
===Running AutoPhrase===
make: Nothing to be done for 'all'.
===RAW_TRAIN: ../../../data/indeed_jobs_extractions/source/corpus.clean.txt===
auto_phrase.sh parameters: indeed_jobs_extractions ../../../data/indeed_jobs_extractions/source/corpus.clean.txt 10 data/EN/wiki_quality.txt 8
===Compilation===
===Tokenization===
Current step: Tokenizing input file...
real	0m1.886s
user	0m12.548s
sys	0m0.736s
Detected Language: EN
Current step: Tokenizing wikipedia phrases...
No provided expert labels.
===Part-Of-Speech Tagging===
===AutoPhrasing===
=== Current Settings ===
Iterations = 2
Minimum Support Threshold = 10
Maximum Length Threshold = 6
POS-Tagging Mode Disabled
Discard Ratio = 0.050000
Number of threads = 8
Labeling Method = DPDN
	Auto labels from knowledge bases
	Max 

## Generate Embeddings

In [56]:
%cd $kb_base_dir/src/concept_learning/

/mnt/efs/shared/meg_shared_scripts/meg-kb/src/concept_learning


In [57]:
!CUDA_VISIBLE_DEVICES=3 python compute_keyphrase_embeddings.py -m bert-base-uncased -et ac -d ../../data/$data_ac/intermediate -c 750

computing entity-wise embedding: 100%|██████| 6853/6853 [04:49<00:00, 23.69it/s]
Saving embedding


# Define Taxonomy

In [58]:
concepts = [
    {
        "alignedCategoryName": "programming_language",
        "unalignedCategoryName": "programming language",
        "generalizations": None,
        "seedInstances": ["python", "sql", "java", "html", "perl", "javascript", "php"]
    },
    {
        "alignedCategoryName": "technology",
        "unalignedCategoryName": "technology",
        "generalizations": None,
        "seedInstances": ["distributed systems", "load balancing", "network monitoring", "data structures", "virtualization technologies", "search engine optimization", "network communications", "version control"]
    },
    {
        "alignedCategoryName": "database_type",
        "unalignedCategoryName": "database type",
        "generalizations": None,
        "seedInstances": ["nosql databases", "oracle", "microsoft sql server", "mongo db", "mysql databases"]
    },
    {
        "alignedCategoryName": "software_development_type",
        "unalignedCategoryName": "software development type",
        "generalizations": None,
        "seedInstances": ["fullstack", "server side", "game development", "enterprise architecture", "mobile application development"]
    },
    {
        "alignedCategoryName": "system_type",
        "unalignedCategoryName": "system type",
        "generalizations": None,
        "seedInstances": ["linux", "android", "unix", "windows"]
    },
    {
        "alignedCategoryName": "job_position",
        "unalignedCategoryName": "job position",
        "generalizations": None,
        "seedInstances": ["architect", "software engineer", "senior software engineer", "junior level", "entry level"]
    },
        {
        "alignedCategoryName": "development_tool",
        "unalignedCategoryName": "development tool",
        "generalizations": None,
        "seedInstances": ["github", "maven", "eclipse", "jenkins", "cuda", "visual studio", "svn"]
    },
]

seed_df = pd.DataFrame(concepts)

In [61]:
# save taxonomy to csv
import os
benchmark_dir = "../../data/indeed_jobs_extractions"
benchmark_filename = 'seed_aligned_concepts.csv'
seed_df.to_csv(os.path.join(benchmark_dir, benchmark_filename), index=None)

# Expand Taxonomy

In [62]:
# add embeddings for any seed instances that have not been already computed
%cd /mnt/efs/shared/meg_shared_scripts/meg-kb/src/concept_learning/
!CUDA_VISIBLE_DEVICES=3 python add_seed_instances_embeddings.py -m bert-base-uncased -et ac -d $benchmark_dir/intermediate -b $benchmark_dir -c 750

/mnt/efs/shared/meg_shared_scripts/meg-kb/src/concept_learning
/home/ubuntu/anaconda3/envs/transformers_nikita/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
Seed instances: ['python', 'sql', 'java', 'html', 'perl', 'javascript', 'php', 'distributed systems', 'load balancing', 'network monitoring', 'data structures', 'virtualization technologies', 'search engine optimization', 'network communications', 'version control', 'nosql databases', 'oracle', 'microsoft sql server', 'mongo db', 'mysql databases', 'fullstack', 'server side', 'game development', 'enterprise architecture', 'mobile application development', 'linux', 'android', 'unix', 'windows', 'architect', 'software engineer', 'senior software engineer', 'junior level', 'entry level', 'github', 'maven', 'eclipse', 'jenkins', 'cuda', 'visual studio', 'svn']
New i

In [68]:
from tqdm.notebook import tqdm
import argparse
import re
import numpy as np
from scipy.spatial.distance import cosine
from scipy.stats import pearsonr, entropy, gmean
import random
import torch
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import BertTokenizer, BertModel, BertForMaskedLM
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import json
from collections import defaultdict, Counter
import time
import importlib

import logging
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression
import pandas as pd
import os
import sys
import math
from annoy import AnnoyIndex
import matplotlib
from matplotlib import pyplot as plt
import networkx as nx

import spacy
from spacy.matcher import Matcher
from spacy.lang.en import English
nlp = English()
nlp.add_pipe(nlp.create_pipe('sentencizer'))
spacy_tokenizer = nlp.tokenizer

nlp_full = spacy.load('en_core_web_sm')

from compute_concept_clusters import knn
from compute_keyphrase_embeddings import ensure_tensor_on_device, mean_pooling

from lm_probes import LMProbe, LMProbe_GPT2, LMProbe_Joint, LMProbe_PMI, LMProbe_PMI_greedy
from utils import load_embeddings, load_seed_aligned_concepts, load_seed_aligned_relations, load_benchmark
from utils import load_EE_labels
from utils import get_masked_contexts, bert_untokenize
from utils import learn_patterns

In [69]:
import utils
importlib.reload(utils)
from utils import load_embeddings, load_seed_aligned_concepts, load_seed_aligned_relations, load_benchmark
from utils import load_EE_labels
from utils import get_masked_contexts, bert_untokenize
from utils import learn_patterns

import lm_probes
importlib.reload(lm_probes)
from lm_probes import LMProbe, LMProbe_GPT2, LMProbe_Joint, LMProbe_PMI, LMProbe_PMI_greedy

In [71]:
bert_emb_path = os.path.join('../../data', data_ac, 'intermediate/BERTembed+seeds.txt')
if not os.path.exists(bert_emb_path):
    bert_emb_path = os.path.join('../../data', data_ac, 'intermediate/BERTembed.txt')
embeddings = load_embeddings(bert_emb_path, 768)
len(embeddings)

6857

## Expand Seed Entities (clustering)
### Seed instances clustering (EE-emb)
(using all seed instances of a concept to find neighbors)

In [73]:
cluster_size = None

!python compute_concept_seeds_knn.py \
-d ../../data/$data_ac/intermediate \
-e ../../data/$data_ac/intermediate/BERTembed+seeds.txt \
-b $benchmark_dir \
-o ../../data/$data_ac/intermediate/ee_concept_knn_k=None.csv \
-kdt

/home/ubuntu/anaconda3/envs/transformers_nikita/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
100%|█████████████████████████████████████████████| 7/7 [00:00<00:00, 22.24it/s]


In [74]:
concept_knn_path = os.path.join('../../data', data_ac, 'intermediate/ee_concept_knn_k=None.csv')
df = pd.read_csv(concept_knn_path)
df[df['concept'] == 'programming_language'].head(10)

,concept,neighbor,sim
0,programming_language,ruby,0.994494
1,programming_language,scripting,0.993874
2,programming_language,jquery,0.992581
3,programming_language,mysql,0.992376
4,programming_language,powershell,0.992175
5,programming_language,xml,0.991677
6,programming_language,css,0.991653
7,programming_language,html5,0.991362
8,programming_language,visual studio,0.991189
9,programming_language,ajax,0.990843


### EE-LM-probe (prompt)


In [75]:
lm_probe = LMProbe(model_name='bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [76]:
!python compute_EE_LM_probe.py \
-d ../../data/$data_ac/intermediate \
-b $benchmark_dir \
-e ../../data/$data_ac/intermediate/BERTembed+seeds.txt \
-lm bert \
-o ../../data/$data_ac/intermediate/ee_LM_bert_k=None.csv

/home/ubuntu/anaconda3/envs/transformers_nikita/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)
lm_probe_type='bert' is deprecated, use 'mlm'
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|█████████████████████████████████████████████| 7/

### MRR

In [77]:
ee_emb_path = os.path.join('../../data', data_ac, 'intermediate/ee_concept_knn_k=None.csv')
ee_LM_path = os.path.join('../../data', data_ac, 'intermediate/ee_LM_bert_k=None.csv')
ee_LM_df = pd.read_csv(ee_LM_path)
ee_emb_df = pd.read_csv(ee_emb_path)
concept_list = ee_LM_df['concept'].drop_duplicates().tolist()

## Using MRR to combine ranking 

ee_mrr_combine_list = []

for _cc in sorted(concept_list):
    _ce_df = ee_emb_df[ee_emb_df['concept'] == _cc].sort_values(by='sim', ascending=False)
    _ee_emb_list = _ce_df['neighbor'].tolist()
    _ee_LM_list = ee_LM_df[ee_LM_df['concept'] == _cc]['neighbor'].tolist()
        
    _all_entities_mrr = defaultdict(float)
    for i, _e in enumerate(_ee_emb_list):
        _all_entities_mrr[_e] += 1.0 / (i+1)
    for i, _e in enumerate(_ee_LM_list):
        _all_entities_mrr[_e] += 1.0 / (i+1)

    _all_entities_mrr_list = sorted(list(_all_entities_mrr.items()), key=lambda p: p[-1], reverse=True)
    
    for _e, _mrr in _all_entities_mrr_list:
        ee_mrr_combine_list.append((_cc, _e, _mrr))

len(ee_mrr_combine_list)

48147

In [79]:
ee_mrr_combine_path = os.path.join('../../data', data_ac, 'intermediate/ee_mrr_combine_bert_k=None.csv')
df = pd.DataFrame(ee_mrr_combine_list, columns=['concept', 'neighbor', 'MRR'])
df = df.merge(ee_LM_df, how='left', on=['concept', 'neighbor'])
df = df.merge(ee_emb_df, how='left', on=['concept', 'neighbor'])
df.to_csv(ee_mrr_combine_path, index=None)
df = pd.read_csv(ee_mrr_combine_path)
mrr = df.groupby('concept').head(200)
mrr.to_csv(os.path.join('../../data', data_ac, 'intermediate/ee_mrr_combine_bert_k=200.csv'), index=None)

In [80]:
for concept, grp in mrr.groupby('concept'):
    print(concept)
    print(grp['neighbor'].tolist()[:20])
    print()
    print()

database_type
['sql', 'oracle database', 'sql server', 'microsoft sql', 'oracle databases', 'mysql', 'java', 'microsoft', 'nosql', 'oracle cloud', 'sql databases', 'postgresql', 'relational databases', 'web services', 'microsoft excel', 'microsoft azure', 'expert level', 'mongodb', 'ms sql', 'server database']


development_tool
['windows', 'selenium', 'java', 'sql', 'adobe flash', 'adobe', 'confluence', 'php', 'mysql', 'visual design', 'subversion', 'microsoft windows', 'bitbucket', 'oracle', 'linux', 'chrome', 'cucumber', 'scripting', 'visual programming', 'python']


job_position
['senior level', 'manager', 'project manager', 'advanced level', 'engineer', 'executive level', 'technical project manager', 'senior project manager', 'professional', 'software developer', 'product manager', 'senior product manager', 'technical lead', 'service level', 'technical program manager', 'program manager', 'product owner', 'phd', 'programmer', 'expert level']


programming_language
['ruby', 'script

In [82]:
lm_preds = []
for concept, grp in ee_LM_df.groupby('concept'):
        grp = grp.reset_index()
        grp = grp.sort_values(by='lm_score', ascending=False).head(200)
        lm_preds.append(grp)
lm_preds = pd.concat(lm_preds)
lm_preds.to_csv(os.path.join('../../data', data_ac, 'intermediate/ee_lm_bert_k=200.csv'), index=None)
lm_preds

for concept, grp in lm_preds.groupby('concept'):
    print(concept)
    print(grp['neighbor'].tolist()[:20])
    print()
    print()

database_type
['oracle database', 'microsoft sql', 'oracle databases', 'sql', 'oracle cloud', 'microsoft', 'sql databases', 'microsoft excel', 'microsoft azure', 'sql server', 'server / database', 'ibm corporation', 'google', 'amazon', 'microsoft outlook', 'google cloud', 'database server', 'ibm db2', 'oracle / linux', 'ibm']


development_tool
['windows', 'java', 'adobe flash', 'adobe', 'php', 'visual design', 'microsoft windows', 'chrome', 'visual programming', 'flash', 'visual language', 'python', 'visual basic', 'maya', 'microsoft excel', 'microsoft project', 'microsoft', 'pdf', 'apache', 'oracle']


job_position
['senior level', 'advanced level', 'executive level', 'senior project manager', 'professional', 'senior product manager', 'technical project manager', 'service level', 'phd', 'technical program manager', 'expert level', 'graduate degree', 'a level', 'master', 'high level', 'technical product manager', 'associate degree', 'advanced degree', 'higher level', 'professional deg

In [84]:
emb_preds = []
for concept, grp in ee_emb_df.groupby('concept'):
        grp = grp.reset_index()
        grp = grp.sort_values(by='sim', ascending=False).head(200)
        emb_preds.append(grp)
emb_preds = pd.concat(emb_preds)
emb_preds.to_csv(os.path.join('../../data', data_ac, 'intermediate/ee_emb_bert_k=200.csv'), index=None)
emb_preds

for concept, grp in emb_preds.groupby('concept'):
    print(concept)
    print(grp['neighbor'].tolist()[:20])
    print()
    print()

database_type
['sql', 'sql server', 'mysql', 'java', 'nosql', 'postgresql', 'relational databases', 'web services', 'expert level', 'mongodb', 'visual studio', 'power bi', 'db2', 'ms sql', 'api', 'microsoft', 'rest api', 'linux', 'selenium', 'pl / sql']


development_tool
['selenium', 'java', 'sql', 'confluence', 'mysql', 'subversion', 'bitbucket', 'cucumber', 'scripting', 'rest api', 'linux', 'oracle', 'sql server', 'web services', 'mvc', 'terraform', 'puppet', 'db2', 'spring boot', 'ide']


job_position
['manager', 'project manager', 'engineer', 'senior level', 'software developer', 'product manager', 'technical lead', 'technical project manager', 'product owner', 'program manager', 'programmer', 'automation engineer', 'member', 'database engineer', 'lead', 'major', 'product development', 'ecommerce', 'product management', 'area']


programming_language
['ruby', 'scripting', 'jquery', 'mysql', 'powershell', 'xml', 'css', 'html5', 'visual studio', 'ajax', 'mvc', 'groovy', 'eclipse', '